In [1]:
import tensorflow as tf
import pandas
import numpy as np
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight

In [3]:
path = os.getcwd()
path = os.path.join(path, "gtsrb-german-traffic-sign")

In [4]:
batch_size = 64
epochs = 30
INIT_LR = 1e-3
IMG_HEIGHT = 32
IMG_WIDTH = 32
numClasses = 43

In [5]:
train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           rotation_range=10, 
                                           zoom_range=0.15,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           shear_range=0.15,
                                           horizontal_flip=False,
                                           vertical_flip=False,
                                           fill_mode="nearest")

test_image_generator = ImageDataGenerator(validation_split=0.8, rescale=1./255)

In [6]:
train_csv = os.path.join(path, "Train.csv")
train_df = pandas.read_csv(train_csv, dtype=str, usecols = ["Path","ClassId"])

In [7]:
train_data_gen = train_image_generator.flow_from_dataframe(
        dataframe=train_df,
        directory=path,
        shuffle=True,
        x_col="Path",
        y_col="ClassId",
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

Found 39209 validated image filenames belonging to 43 classes.


In [8]:
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_data_gen.classes), 
                train_data_gen.classes)

In [9]:
test_csv = os.path.join(path, "Test.csv")
test_df = pandas.read_csv(test_csv, dtype=str, usecols = ["Path","ClassId"])

In [10]:
val_data_gen = test_image_generator.flow_from_dataframe(
        dataframe=test_df,
        directory=path,
        subset='validation',
        x_col="Path",
        y_col="ClassId",
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

test_data_gen = test_image_generator.flow_from_dataframe(
        dataframe=test_df,
        directory=path,
        subset='training',
        x_col="Path",
        y_col="ClassId",
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

Found 10104 validated image filenames belonging to 43 classes.
Found 2526 validated image filenames belonging to 43 classes.


In [11]:
model = Sequential([
    Conv2D(8, 5, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(16, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(numClasses, activation='softmax')
])

In [12]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / (epochs * 0.5))

In [13]:
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [14]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=500,
    epochs=epochs,
    validation_data=val_data_gen,
    class_weight=class_weights,
    verbose = 1
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 158 steps
Epoch 1/30
500/500 [==============================] - 59s 118ms/step - loss: 2.9309 - accuracy: 0.2391 - val_loss: 1.9536 - val_accuracy: 0.3712
Epoch 2/30
500/500 [==============================] - 64s 127ms/step - loss: 1.7053 - accuracy: 0.4545 - val_loss: 1.1392 - val_accuracy: 0.5993
Epoch 3/30
500/500 [==============================] - 66s 131ms/step - loss: 1.2119 - accuracy: 0.5991 - val_loss: 0.7466 - val_accuracy: 0.7537
Epoch 4/30
500/500 [==============================] - 68s 136ms/step - loss: 0.8586 - accuracy: 0.7168 - val_loss: 0.5648 - val_accuracy: 0.8057
Epoch 5/30
500/500 [==============================] - 68s 136ms/step - loss: 0.6417 - accuracy: 0.7937 - val_loss: 0.3872 - val_accuracy: 0.8749
Epoch 6/30
500/500 [==============================] - 66s 131ms/step - loss: 0.5119 - accuracy: 0.8338 - 

In [15]:
test_loss, test_accuracy = model.evaluate(test_data_gen)

  ...
    to  
  ['...']
40/40 [==============================] - 1s 36ms/step - loss: 0.1511 - accuracy: 0.9600


In [16]:
model.save("TrafficSignRec")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: TrafficSignRec\assets
